# Week 3: Segmenting and Clustering Neighborhoods in Toronto

### 1. Loading libraries

In [19]:
# loading libraries from Lab's notebook

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install -c beautifulsoup4
from bs4 import BeautifulSoup
import requests

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

### 2. Scrapping list of neighborhoods from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [20]:
r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
data = r.text
soup = BeautifulSoup(data)
soup = BeautifulSoup(str(soup).replace("\n", ""))

In [21]:
Postal = []
Borough = []
Neighbourhood = []

for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        Postal.append(data[0].text)
        Borough.append(data[1].text)
        Neighbourhood.append(data[2].text)
    except IndexError:pass


neighbourhood = pd.DataFrame(pd.DataFrame(data={
    'Postal' : Postal,
    'Borough' : Borough,
    'Neighborhood' : Neighbourhood
    }))

### 3. Cleaning and organizing the data

In [22]:
neighbourhood['Neighborhood'].replace("Not assigned", neighbourhood["Borough"],inplace=True)
neighbourhood = neighbourhood[neighbourhood.Borough != 'Not assigned']
neighbourhood = neighbourhood.groupby(['Postal', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

### 4. Printing head and shape of dataframe

In [23]:
neighbourhood.head(10)

,Postal,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"
